##### <b>Step-1: Importing Libraries</b>

In [22]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf
from PIL import Image
import numpy as np

import tensorflow as tf
from tensorflow import keras
from skimage import transform
from google.colab import drive
from keras import backend as K
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from keras.models import load_model
from keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,Activation,Conv2DTranspose,MaxPool2D,Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##### <b>Step-2: Mounting Drive</b>

In [23]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
path="/content/drive/MyDrive/CRACK500-Resized"

In [25]:
os.listdir(path)

['pytorch-road-segmentation-syncrack.ipynb',
 'deeplabv3_plus.ipynb',
 'Test',
 'val',
 'Train',
 'Wound_Segmentation_on_Fuseg_&_Medec_Data.ipynb',
 'segmentation.ipynb',
 'pneumonia_detection_using_Unet.ipynb']

In [26]:
train_images_path=os.path.join(os.path.join(path,'Train'),'images')
train_mask_path=os.path.join(os.path.join(path,'Train'),'masks')

In [27]:
test_images_path=os.path.join(os.path.join(path,'Test'),'images')
test_mask_path=os.path.join(os.path.join(path,'Test'),'masks')

In [28]:
val_images_path=os.path.join(os.path.join(path,'val'),'images')
val_mask_path=os.path.join(os.path.join(path,'val'),'masks')

In [29]:
train_ids = next(os.walk(train_images_path))[1]

In [30]:
len(os.listdir(test_images_path))

1124

In [31]:
import glob

In [32]:
import os
import cv2
import numpy as np
from random import shuffle
def read_data(images_path, masks_path, ext, sign):
    images = []
    masks = []
    i = 0

    for subdir, dirs, files in os.walk(images_path):
        for file in files:
            name = os.path.splitext(file)[0]
            image = cv2.imread(os.path.join(images_path, file))
            mask_file = file.replace(".jpg", ext)
            mask = cv2.imread(os.path.join(masks_path, mask_file))

            # Resize the image and mask
            image = cv2.resize(image, (128,128))
            mask = cv2.resize(mask, (128,128))

            # Convert the mask to grayscale
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

            # Expand the dimensions of the mask
            mask = np.expand_dims(mask, axis=-1)

            # Apply image augmentations
            images.append(image/255.0)
            masks.append(mask/255.0)

            if sign == 1:
                # Randomly flip horizontally
                image_aug = cv2.flip(image, 1)
                mask_aug = cv2.flip(mask, 1)
                mask_aug = np.expand_dims(mask_aug, axis=-1)
                images.append(image_aug/255.0)
                masks.append(mask_aug/255.0)

                # Randomly flip vertically
                image_aug = cv2.flip(image, 0)
                mask_aug = cv2.flip(mask, 0)
                mask_aug = np.expand_dims(mask_aug, axis=-1)
                images.append(image_aug/255.0)
                masks.append(mask_aug/255.0)

                # Random rotation
                angle = np.random.uniform(low=-10, high=10)  # Random angle between -10 and 10 degrees
                rows, cols = image_aug.shape[:2]
                M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
                image_aug = cv2.warpAffine(image_aug, M, (cols, rows))
                mask_aug = cv2.warpAffine(mask_aug, M, (cols, rows))
                mask_aug = np.expand_dims(mask_aug, axis=-1)
                images.append(image_aug/255.0)
                masks.append(mask_aug/255.0)

                # Add noise
                noise = np.random.normal(loc=0, scale=0.1, size=image.shape)  # Gaussian noise with mean 0 and standard deviation 0.1
                image_aug = image + noise
                images.append(np.clip(image, 0, 255) / 255.0)
                masks.append(mask / 255.0)

                # Random translation
                tx = np.random.uniform(low=-5, high=5)  # Random translation in the x-direction between -5 and 5 pixels
                ty = np.random.uniform(low=-5, high=5)  # Random translation in the y-direction between -5 and 5 pixels
                M = np.float32([[1, 0, tx], [0, 1, ty]])
                image_aug = cv2.warpAffine(image, M, (cols, rows))
                mask_aug = cv2.warpAffine(mask, M, (cols, rows))
                mask_aug = np.expand_dims(mask_aug, axis=-1)
                images.append(image_aug / 255.0)
                masks.append(mask_aug / 255.0)

                # Brightness and Contrast Adjustment
                alpha = np.random.uniform(low=0.7, high=1.3)  # Random scaling factor for contrast adjustment
                beta = np.random.uniform(low=-0.1, high=0.1)  # Random shift for brightness adjustment
                image_aug = np.clip(image * alpha + beta, 0, 255)
                images.append(image_aug / 255.0)
                masks.append(mask / 255.0)

                # Gamma Correction
                gamma = np.random.uniform(low=0.8, high=1.2)  # Random gamma value
                image_aug = np.clip(image_aug ** gamma, 0, 255)
                images.append(image_aug / 255.0)
                masks.append(mask / 255.0)

            if i == 500:
                break

            if i % 100 == 0:
                print(i)

            i += 1
            if sign == 0 and i == 600:
                break

    # Shuffle the images and masks
    combined = list(zip(images, masks))
    shuffle(combined)
    images, masks = zip(*combined)
    images = np.array(images)
    masks = np.array(masks)

    return images, masks


In [33]:
train_images,train_masks=read_data(train_images_path,train_mask_path,".png",1)


0
100
200
300
400


In [34]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 4.4 MB/s eta 0:00:00
albumentations==1.3.1 is successfully installed


In [35]:
# import os
# import cv2
# import numpy as np
# import albumentations as A

# def read_data(images_path, masks_path, ext):
#     images = []
#     masks = []
#     i = 0

#     for subdir, dirs, files in os.walk(images_path):
#         for file in files:
#             name = os.path.splitext(file)[0]
#             image = cv2.imread(os.path.join(images_path, file))
#             mask_file = file.replace(".jpg", ext)
#             mask = cv2.imread(os.path.join(masks_path, mask_file))

#             # Resize the image and mask
#             image = cv2.resize(image, (128,128), interpolation=cv2.INTER_AREA)
#             mask = cv2.resize(mask, (128,128), interpolation=cv2.INTER_AREA)

#             # Convert the mask to grayscale
#             mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

#             # Expand the dimensions of the mask
#             mask = np.expand_dims(mask, axis=-1)

#             # Append the original image and mask to the lists
#             images.append(image/255.0)
#             masks.append(mask/255.0)

#             # if ext == ".png":
#             #     # Define the image augmentation transformations
#             #     transform = A.Compose([
#             #         A.HorizontalFlip(p=0.5),
#             #         A.VerticalFlip(p=0.5),
#             #         A.Rotate(limit=15, p=0.5),
#             #         A.RandomBrightnessContrast(p=0.5),
#             #         A.Blur(blur_limit=3, p=0.5),
#             #         A.RandomGamma(p=0.5),
#             #         A.RandomBrightness(p=0.5),
#             #         A.RandomContrast(p=0.5),
#             #         # Add more augmentations as needed
#             #     ])

#             #     # Apply the augmentations to the image and mask
#             #     augmented = transform(image=image, mask=mask)
#             #     image_aug = augmented["image"]
#             #     mask_aug = augmented["mask"]

#             #     # # Expand the dimensions of the mask
#             #     # mask_aug = np.expand_dims(mask_aug, axis=-1)

#             #     # Normalize the image and mask
#             #     image_aug = image_aug / 255.0
#             #     mask_aug = mask_aug / 255.0

#             #     # Append the augmented image and mask to the lists
#             #     images.append(image_aug)
#             #     masks.append(mask_aug)

#             if i % 20 == 0:
#                 print(i)
#             i = i + 1

#     images = np.array(images)
#     masks = np.array(masks)

#     return images, masks


In [36]:
train_images,train_masks=read_data(train_images_path,train_mask_path,".png",1)


0
100
200
300
400


In [37]:
# val_images,val_masks=read_data(val_images_path,val_mask_path,"_mask.png",1)

In [38]:
# import numpy as np

# # Assuming you have the train_images, train_masks, val_images, and val_masks arrays

# train_images = np.concatenate((train_images, val_images), axis=0)

# train_masks = np.concatenate((train_masks, val_masks), axis=0)

# # Print the shape of the combined data
# print("Train data shape:", train_images.shape)
# print("Test data shape:", train_masks.shape)


In [39]:
val_images,val_masks=read_data(test_images_path,test_mask_path,".png",0)

0
100
200
300
400


In [40]:
# val_images,val_masks=read_data(val_images_path,val_mask_path,"_mask.png")

In [41]:
train_images.shape,val_images.shape,train_masks.shape, val_masks.shape

((4008, 128, 128, 3),
 (501, 128, 128, 3),
 (4008, 128, 128, 1),
 (501, 128, 128, 1))

In [42]:
image_size=224

# Unet Model

## Encoder and Decoder

In [43]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [44]:
def UNet():
    f = [32, 64, 128, 256,512]
    inputs = keras.layers.Input((image_size, image_size, 3))

    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8

    bn = bottleneck(p4, f[4])

    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128

    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [45]:
import tensorflow as tf

# Define custom IoU metric function
def iou_score(y_true, y_pred):
    y_true = tf.cast(y_true > 0.5, dtype=tf.float32)
    y_pred = tf.cast(y_pred > 0.5, dtype=tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    iou = intersection / union
    return iou
def jaccard_distance(y_true, y_pred, smooth=100):
    """ Calculates mean of Jaccard distance as a loss function """
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1,2))
    sum_ = tf.reduce_sum(y_true + y_pred, axis=(1,2))
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    jd =  (1 - jac) * smooth
    return tf.reduce_mean(jd)

In [46]:
# import tensorflow as tf
# import tensorflow.keras.layers as L
# from tensorflow.keras.models import Model

# # Define precision and recall as metrics
# precision = tf.keras.metrics.Precision()
# recall = tf.keras.metrics.Recall()

# def conv_block(x, num_filters):
#     x = L.Conv2D(num_filters, 3, padding="same")(x)
#     x = L.BatchNormalization()(x)
#     x = L.Activation("relu")(x)

#     x = L.Conv2D(num_filters, 3, padding="same")(x)
#     x = L.BatchNormalization()(x)
#     x = L.Activation("relu")(x)

#     return x

# def encoder_block(x, num_filters):
#     x = conv_block(x, num_filters)
#     p = L.MaxPool2D((2, 2))(x)
#     return x, p

# def attention_gate(g, s, num_filters):
#     Wg = L.Conv2D(num_filters, 1, padding="same")(g)
#     Wg = L.BatchNormalization()(Wg)

#     Ws = L.Conv2D(num_filters, 1, padding="same")(s)
#     Ws = L.BatchNormalization()(Ws)

#     out = L.Activation("relu")(Wg + Ws)
#     out = L.Conv2D(num_filters, 1, padding="same")(out)
#     out = L.Activation("sigmoid")(out)

#     return out * s

# def decoder_block(x, s, num_filters):
#     x = L.UpSampling2D(interpolation="bilinear")(x)
#     s = attention_gate(x, s, num_filters)
#     x = L.Concatenate()([x, s])
#     x = conv_block(x, num_filters)
#     return x

# def attention_unet(input_shape):
#     """ Inputs """
#     inputs = L.Input(input_shape)

#     """ Encoder """
#     s1, p1 = encoder_block(inputs, 64)
#     s2, p2 = encoder_block(p1, 128)
#     s3, p3 = encoder_block(p2, 256)

#     b1 = conv_block(p3, 512)

#     """ Decoder """
#     d1 = decoder_block(b1, s3, 256)
#     d2 = decoder_block(d1, s2, 128)
#     d3 = decoder_block(d2, s1, 64)

#     """ Outputs """
#     outputs = L.Conv2D(1, 1, padding="same", activation="sigmoid")(d3)

#     """ Model """
#     model = Model(inputs, outputs, name="Attention-UNET")
#     return model
# input_shape = (224,224, 3)
# model = attention_unet(input_shape)

# model.compile(optimizer="adam", loss=jaccard_distance, metrics=[precision,recall,iou_score])
# model.summary()

In [47]:
!pip3 install -U segmentation-models
%env SM_FRAMEWORK=tf.keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00
env: SM_FRAMEWORK=tf.keras


In [48]:
import tensorflow as tf

def f1_score(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1)))
    actual_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))

    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (actual_positives + tf.keras.backend.epsilon())

    f1 = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
    return f1


In [49]:
import segmentation_models as sm
# Compile the model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

def dice_coefficient(y_true, y_pred, smooth=1e-7):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return dice

# binary segmentation (these parameters are default when you call Unet('resnet34')
model = sm.Unet('resnet34',input_shape=(128,128,3),encoder_weights="imagenet", classes=1, activation='sigmoid')
preprocess_input = sm.get_preprocessing('resnet34')
train_images = preprocess_input(train_images)
val_images = preprocess_input(val_images)

# Define the optimizer
optimizer = Adam(lr=0.0001)
model.compile(
    optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy", sm.metrics.iou_score, dice_coefficient,f1_score],
)

lr_scheduler = ReduceLROnPlateau(factor=0.1, patience=3, min_lr=1e-6)
model.summary()


Segmentation Models: using `tf.keras` framework.
85521592/85521592 [==============================] - 0s 0us/step


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 128, 128, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0          ['bn_data[0][0]']                
                                                                                                  
 conv0 (Conv2D)                 (None, 64, 64, 64)   9408        ['zero_padding2d[0][0]']   

# Model Compilation

In [50]:
# # # Define precision and recall as metrics
# import tensorflow as tf
# precision = tf.keras.metrics.Precision()
# recall = tf.keras.metrics.Recall()
# def dice_coef(groundtruth_mask, pred_mask):
#     intersect = np.sum(pred_mask*groundtruth_mask)
#     total_sum = np.sum(pred_mask) + np.sum(groundtruth_mask)
#     dice = np.mean(2*intersect/total_sum)
#     return round(dice, 3) #round up to 3 decimal places
# model = UNet()
# model.compile(optimizer="adam", loss=jaccard_distance, metrics=[precision,recall,dice_coef,iou_score])
# model.summary()

In [ ]:
history = model.fit(train_images,train_masks,
                          batch_size=4,epochs=20,verbose = 1,validation_data=(val_images,val_masks)
                          )

Epoch 1/20
1002/1002 [==============================] - 91s 44ms/step - loss: 0.1243 - accuracy: 0.9429 - iou_score: 0.3468 - dice_coefficient: 0.5057 - f1_score: 0.6035 - val_loss: 0.5850 - val_accuracy: 0.9399 - val_iou_score: 0.1023 - val_dice_coefficient: 0.1774 - val_f1_score: 0.1255
Epoch 2/20
1002/1002 [==============================] - 42s 42ms/step - loss: 0.0987 - accuracy: 0.9509 - iou_score: 0.4255 - dice_coefficient: 0.5929 - f1_score: 0.6869 - val_loss: 0.0842 - val_accuracy: 0.9574 - val_iou_score: 0.3855 - val_dice_coefficient: 0.5532 - val_f1_score: 0.6283
Epoch 3/20
1002/1002 [==============================] - 42s 42ms/step - loss: 0.0893 - accuracy: 0.9541 - iou_score: 0.4618 - dice_coefficient: 0.6279 - f1_score: 0.7159 - val_loss: 0.0874 - val_accuracy: 0.9579 - val_iou_score: 0.4083 - val_dice_coefficient: 0.5755 - val_f1_score: 0.6428
Epoch 4/20
1002/1002 [==============================] - 43s 43ms/step - loss: 0.0863 - accuracy: 0.9550 - iou_score: 0.4714 - dice

In [ ]:
import matplotlib.pyplot as plt
import math

# Predict the masks for validation images
pred_masks = model.predict(val_images)

# Define the number of rows and columns for displaying the images
num_images = 3

# Select random indices for displaying the images
indices = np.random.choice(len(val_images), size=num_images, replace=False)

# Create subplots for displaying the images
fig, axes = plt.subplots(num_images, 3, figsize=(12, 12))

# Iterate over the selected indices and display the images
for i, index in enumerate(indices):
    # Display the original image
    axes[i, 0].imshow(val_images[index])
    axes[i, 0].set_title('Original Image')
    axes[i, 0].axis('off')

    # Display the predicted mask
    axes[i, 1].imshow(pred_masks[index], cmap='gray')
    axes[i, 1].set_title('Predicted Mask')
    axes[i, 1].axis('off')

    # Display the validation mask
    axes[i, 2].imshow(val_masks[index], cmap='gray')
    axes[i, 2].set_title('Validation Mask')
    axes[i, 2].axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the figure
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot metrics
plt.figure(figsize=(12, 8))

# Loss
plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Precision
plt.subplot(2, 2, 2)
plt.plot(history.history['precision'], label='Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()

# Recall
plt.subplot(2, 2, 3)
plt.plot(history.history['recall'], label='Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()

# IoU
plt.subplot(2, 2, 4)
plt.plot(history.history['iou_score'], label='IoU')
plt.plot(history.history['val_iou_score'], label='Validation IoU')
plt.xlabel('Epoch')
plt.ylabel('IoU')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Plot metrics
plt.figure(figsize=(12, 6))

# Precision
plt.plot(history.history['precision'], label='Precision')
plt.plot(history.history['val_precision'], label='Validation Precision')

# Recall
plt.plot(history.history['recall'], label='Recall')
plt.plot(history.history['val_recall'], label='Validation Recall')

plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Precision and Recall')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(masks[0]*255, (image_size, image_size)), cmap="gray")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(result[0]*255, (image_size, image_size)), cmap="gray")